In [1]:
import json
import math
import uuid
import random
import pandas as pd
from datetime import datetime
import fhirclient.r3.models.patient as p
import fhirclient.r3.models.humanname as hn
import fhirclient.r3.models.observation as o
import fhirclient.r3.models.quantity as q
import fhirclient.r3.models.coding as c
import fhirclient.r3.models.codeableconcept as cc
import fhirclient.r3.models.fhirdate as dt
import fhirclient.r3.models.bundle as b
import fhirclient.r3.models.fhirreference as fr
import fhirclient.r3.models.identifier as iden

from faker import Faker

fake = Faker()

file = "loinc.csv"

df = pd.read_csv(file)
df

,Code,Display,ValueType,value,q.unit,q.system,q.code,cc.code,cc.system,cc.display
0,30180-4,Basophils/100 leukocytes in Blood,Quantity,1.0,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN
1,26449-9,Eosinophils [#/volume] in Blood,Quantity,0.5,thousands per microliter,http://unitsofmeasure.org,10*3/uL,NaN,NaN,NaN
2,718-7,Hemoglobin [Mass/volume] in Blood,Quantity,15.0,gram per deciliter,http://unitsofmeasure.org,g/dL,NaN,NaN,NaN
3,26464-8,Leukocytes [#/volume] in Blood,Quantity,5.2,billion per liter,http://unitsofmeasure.org,10*9/L,NaN,NaN,NaN
4,40651-2,Metamyelocytes [Presence] in Blood,CodeableConcept,NaN,NaN,NaN,NaN,LA9633-4,http://loinc.org,Present
5,74398-9,Neutrophils [#/volume] in Cord blood,Quantity,7.9,billion per liter,http://unitsofmeasure.org,10*9/L,NaN,NaN,NaN


In [3]:
patient = p.Patient()
gender = random.choice(['male','female'])
patient.gender = gender
name = hn.HumanName()
if gender == 'male':
    given_name = fake.first_name_male()
elif gender == 'female':
    given_name = fake.first_name_female()
family_name = fake.last_name()
name.family = family_name
name.given = [given_name, "Dash"]
name.text = given_name + " Dash " + family_name
name.use = 'official'
patient.name = [name]
birthdate = dt.FHIRDate()
birthdate.date = fake.date_between(start_date='-80y', end_date='-6y')
patient.birthDate = birthdate
identifier = iden.Identifier()
identifier.system = "http://example.org"
identifier.value = uuid.uuid4().hex
patient.identifier = [identifier]
patient.uuid = uuid.uuid4().urn
print(json.dumps(patient.as_json(), indent=4))
print(patient.uuid)

{
    "birthDate": "1992-09-17",
    "gender": "male",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "aa2ddb7d881c4c90a65866c7937c48f1"
        }
    ],
    "name": [
        {
            "family": "Sanders",
            "given": [
                "Edward",
                "Dash"
            ],
            "text": "Edward Dash Sanders",
            "use": "official"
        }
    ],
    "resourceType": "Patient"
}
urn:uuid:597202e9-b8b3-479e-8de4-4e205f7f4702


In [4]:
bundle = b.Bundle()
bundle.type = 'transaction'
bundleEntry = b.BundleEntry()
bundleEntry.fullUrl = patient.uuid
bundleEntry.resource = patient
bundleEntryRequest = b.BundleEntryRequest()
bundleEntryRequest.method = "POST"
bundleEntryRequest.url = "Patient"
bundleEntry.request = bundleEntryRequest
bundle.entry = [bundleEntry]
print(json.dumps(bundle.as_json(), indent=4))

{
    "entry": [
        {
            "fullUrl": "urn:uuid:597202e9-b8b3-479e-8de4-4e205f7f4702",
            "request": {
                "method": "POST",
                "url": "Patient"
            },
            "resource": {
                "birthDate": "1992-09-17",
                "gender": "male",
                "identifier": [
                    {
                        "system": "http://example.org",
                        "value": "aa2ddb7d881c4c90a65866c7937c48f1"
                    }
                ],
                "name": [
                    {
                        "family": "Sanders",
                        "given": [
                            "Edward",
                            "Dash"
                        ],
                        "text": "Edward Dash Sanders",
                        "use": "official"
                    }
                ],
                "resourceType": "Patient"
            }
        }
    ],
    "type": "transaction",
    "r

In [5]:
subject = fr.FHIRReference()
subject.reference = patient.uuid
print(json.dumps(subject.as_json(), indent=4))

{
    "reference": "urn:uuid:597202e9-b8b3-479e-8de4-4e205f7f4702"
}


In [6]:

# for index, row in df.iterrows():
#     if row['ValueType'] == 'Quantity':
#         print(row["Code"])
#         print(row["Display"])
#         print(row["value"])
#         value = row["value"]
#         low = value*0.25
#         print("min = ", low)
#         high = value*1.75
#         print("max = ", high)
#         print("newvalue = ", math.ceil(random.uniform(low, high)*10)/10)
#         print(" ")

# value = 5
# if isinstance(value, int):
#     low = int(round(value*0.25))
#     print("min = ", low)
#     high = int(round(value*1.75))
#     print("max = ", high)
#     print("newvalue = ", random.randrange(low, high))
#     print(" ")
# elif isinstance(value, float):
#     low = value*0.25
#     print("min = ", low)
#     high = value*1.75
#     print("max = ", high)
#     print("newvalue = ", math.ceil(random.uniform(low, high)*10)/10)

# for i in range(1,10):
#     print(math.ceil(random.uniform(1.0, 100.0)*10)/10)
    

In [7]:

obslist = []
for index, row in df.iterrows():
    if row['ValueType'] == 'Quantity':
        # for each LOINC code
        for i in range(random.randint(1, 5)):
            # make 1 to 10 examples
            observation = o.Observation()
            observation.status = "final"
        
            code = cc.CodeableConcept()
            codecode = c.Coding()
            codecode.system = 'http://loinc.org'
            codecode.code = row['Code']
            codecode.display = row['Display']
            code.text = row['Display']
            code.coding = [codecode]
            observation.code = code

            category = cc.CodeableConcept()
            categorycoding = c.Coding()
            categorycoding.system = "http://loinc.org"
            categorycoding.code = "laboratory"
            categorycoding.display = "Laboratory"
            category.coding = [categorycoding]
            observation.category = [category]

            observation.subject = subject
        
            # print(type(row['value']))
            
            effectiveDateTime = dt.FHIRDate()
            effectiveDateTime.date = fake.date_time_between(start_date=birthdate.date, end_date='now')
            # effectiveDateTime.date = fake.date_time_between(start_date='-10y', end_date='now')
            observation.effectiveDateTime = effectiveDateTime
            
            value = row['value']
            low = value * 0.1
            high = value * 2
            newvalue = math.ceil(random.uniform(low, high)*10)/10
#             if value < 10.0:
#                 newvalue = math.ceil(random.uniform(0.1, 10.0)*10)/10
#             else:
#                 newvalue = math.ceil(random.uniform(1.0, 100.0)*10)/10
            # print(newvalue)
            # newvalue = 1.5
            valuequantity = q.Quantity()
            valuequantity.value = newvalue
            # print(valuequantity.value)
            valuequantity.unit = row['q.unit']
            valuequantity.system = row['q.system']
            valuequantity.code = row['q.code']
            # print(json.dumps(valuequantity.as_json(), indent=4))
            observation.valueQuantity = valuequantity
            
            # print(json.dumps(observation.as_json(), indent=4))
            bundleEntry = b.BundleEntry()
            bundleEntryRequest = b.BundleEntryRequest()
            bundleEntryRequest.method = "POST"
            bundleEntryRequest.url = "Observation"
            bundleEntry.request = bundleEntryRequest
            bundleEntry.resource = observation
            # myuuid = uuid.uuid4().urn
            bundleEntry.fullUrl = uuid.uuid4().urn
            # print(json.dumps(bundleEntry.as_json(), indent=4))
            # obslist.append(bundleEntry)
            bundle.entry.append(bundleEntry)      
# bundle.entry.append(obslist)
print(json.dumps(bundle.as_json(), indent=4))

{
    "entry": [
        {
            "fullUrl": "urn:uuid:597202e9-b8b3-479e-8de4-4e205f7f4702",
            "request": {
                "method": "POST",
                "url": "Patient"
            },
            "resource": {
                "birthDate": "1992-09-17",
                "gender": "male",
                "identifier": [
                    {
                        "system": "http://example.org",
                        "value": "aa2ddb7d881c4c90a65866c7937c48f1"
                    }
                ],
                "name": [
                    {
                        "family": "Sanders",
                        "given": [
                            "Edward",
                            "Dash"
                        ],
                        "text": "Edward Dash Sanders",
                        "use": "official"
                    }
                ],
                "resourceType": "Patient"
            }
        },
        {
            "fullUrl": "urn:u

In [25]:
with open('loinc_bundle.json', 'w') as outfile:
    json.dump(bundle.as_json(), outfile, indent=2)

In [8]:
uuid.uuid4().urn

'urn:uuid:be4cc3f4-2799-47b9-9350-8ff924036a2a'

In [9]:
fake = Faker()

In [10]:
fake.name()

'David Johnson'

In [11]:
fake.address()

'025 Katie Plains\nRollinsview, VT 53493'

In [12]:
for _ in range(10):
    print(fake.name())

Mr. Joseph Dougherty
Dr. Cheryl Henderson
Lisa Hill
Jennifer Lee
Francis Castro
Michael Jimenez
Christopher Choi II
Nicole Fowler
Lawrence Mccann
Edward Jackson


In [13]:
fake.date_time_between(start_date='-5y', end_date='now').isoformat()

'2020-11-17T14:27:40'

In [14]:
fakedate = fake.date_time_between(start_date='-10y', end_date='now').isoformat()
print(fakedate)

2014-04-29T10:40:12


In [15]:
edt = dt.FHIRDate()
# print(fakedate)
# edt.date = datetime.fromisoformat('2011-11-04T00:05:23+00:00')
edt = datetime.fromisoformat(fakedate)
print(edt)

2014-04-29 10:40:12


In [16]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2021-07-08T08:04:50.951799


In [17]:
myfakedate = fake.date_time_between(start_date='-5y', end_date='now').isoformat()
myfakedate
datetime.fromisoformat(myfakedate).isoformat()

'2019-06-12T09:36:28'

In [18]:
import random
my_list = ['A'] * 20 + ['B'] * 20 + ['C'] * 60
random.choice(my_list)

'C'

In [20]:
mybun = b.Bundle()
mybun.type = 'transaction'
mybun.entry = []
for i in range(1,5):
    myobs = o.Observation()
    observation = o.Observation()
    observation.status = "final"

    code = cc.CodeableConcept()
    codecode = c.Coding()
    codecode.system = 'http://loinc.org'
    codecode.code = '30180-4'
    code.coding = [codecode]
    observation.code = code

    valuequantity = q.Quantity()
    valuequantity.value = i
    observation.valueQuantity = valuequantity
    
    bundleEntry = b.BundleEntry()
    bundleEntryRequest = b.BundleEntryRequest()
    bundleEntryRequest.method = "POST"
    bundleEntryRequest.url = "Observation"
    bundleEntry.request = bundleEntryRequest
    bundleEntry.resource = observation
    # print(json.dumps(bundleEntry.as_json(), indent=4))
    mybun.entry.append(bundleEntry)
    # print(type(mybun.entry))
    
print(json.dumps(mybun.as_json(), indent=4))
    

{
    "entry": [
        {
            "request": {
                "method": "POST",
                "url": "Observation"
            },
            "resource": {
                "code": {
                    "coding": [
                        {
                            "code": "30180-4",
                            "system": "http://loinc.org"
                        }
                    ]
                },
                "status": "final",
                "valueQuantity": {
                    "value": 1
                },
                "resourceType": "Observation"
            }
        },
        {
            "request": {
                "method": "POST",
                "url": "Observation"
            },
            "resource": {
                "code": {
                    "coding": [
                        {
                            "code": "30180-4",
                            "system": "http://loinc.org"
                        }
                    ]
       

In [21]:
gender = random.choice(['male','female'])
print(gender)

male


In [22]:
birthdate = fake.date()
print(birthdate)

2002-12-09


In [23]:
uuid.uuid4().hex

'3c12d477f3a145c4a5c1bf293b223850'